In [63]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [64]:
cyclists_data = pd.read_csv("dataset/cyclists.csv",index_col=0)
races_data = pd.read_csv("dataset/races.csv",index_col=0)
cyclists_data.info()
cyclists_data.head()

<class 'pandas.core.frame.DataFrame'>
Index: 6134 entries, 0 to 6133
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         6134 non-null   object 
 1   birth_year   6121 non-null   float64
 2   weight       3078 non-null   float64
 3   height       3143 non-null   float64
 4   nationality  6133 non-null   object 
 5   cyclist_id   6134 non-null   object 
dtypes: float64(3), object(3)
memory usage: 335.5+ KB


,name,birth_year,weight,height,nationality,cyclist_id
0,Bruno Surra,1964.0,NaN,NaN,Italy,bruno-surra
1,Gérard Rué,1965.0,74.0,182.0,France,gerard-rue
2,Jan Maas,1996.0,69.0,189.0,Netherlands,jan-maas
3,Nathan Van Hooydonck,1995.0,78.0,192.0,Belgium,nathan-van-hooydonck
4,José Félix Parra,1997.0,55.0,171.0,Spain,jose-felix-parra


In [65]:
races_data.info()
races_data.head()

<class 'pandas.core.frame.DataFrame'>
Index: 589865 entries, 0 to 589864
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   _url                 589865 non-null  object 
 1   name                 589865 non-null  object 
 2   points               589388 non-null  float64
 3   uci_points           251086 non-null  float64
 4   length               589865 non-null  float64
 5   climb_total          442820 non-null  float64
 6   profile              441671 non-null  float64
 7   startlist_quality    589865 non-null  int64  
 8   average_temperature  29933 non-null   float64
 9   date                 589865 non-null  object 
 10  position             589865 non-null  int64  
 11  cyclist_age          589752 non-null  float64
 12  is_tarmac            589865 non-null  bool   
 13  is_cobbled           589865 non-null  bool   
 14  is_gravel            589865 non-null  bool   
 15  cyclist_team         4

,_url,name,points,uci_points,length,climb_total,profile,startlist_quality,average_temperature,date,position,cyclist_age,is_tarmac,is_cobbled,is_gravel,cyclist_team,delta,cyclist_id
0,tour-de-france/1978/stage-6,Tour de France,100.0,NaN,162000.0,1101.0,1.0,1241,NaN,1978-07-05 04:02:24,0,22.0,True,False,False,vini-ricordi-pinarello-sidermec-1986,0.0,sean-kelly
1,tour-de-france/1978/stage-6,Tour de France,100.0,NaN,162000.0,1101.0,1.0,1241,NaN,1978-07-05 04:02:24,1,27.0,True,False,False,norway-1987,0.0,gerrie-knetemann
2,tour-de-france/1978/stage-6,Tour de France,100.0,NaN,162000.0,1101.0,1.0,1241,NaN,1978-07-05 04:02:24,2,24.0,True,False,False,NaN,0.0,rene-bittinger
3,tour-de-france/1978/stage-6,Tour de France,100.0,NaN,162000.0,1101.0,1.0,1241,NaN,1978-07-05 04:02:24,3,30.0,True,False,False,navigare-blue-storm-1993,0.0,joseph-bruyere
4,tour-de-france/1978/stage-6,Tour de France,100.0,NaN,162000.0,1101.0,1.0,1241,NaN,1978-07-05 04:02:24,4,27.0,True,False,False,spain-1991,0.0,sven-ake-nilsson


# 1. Data Processing

## 1.1 Categorical variables,  normalizing and standarizing

In [60]:
# Define function to categorize 'position'
# This will later be the label for our classifiers
def categorize_position(pos):
    if pos <= 3:
        return "Podium"
    elif pos <= 20:
        return "Top20"
    elif pos <= 50:
        return "Top50"
    elif pos <= 100:
        return "Top100"
    else:
        return "Remaining"

# Apply categorization
races_data["position_category"] = races_data["position"].apply(categorize_position)

# Check distribution of categories
position_distribution = races_data["position_category"].value_counts(normalize=True) * 100

position_distribution


position_category
Remaining    31.518737
Top100       31.315301
Top50        20.382121
Top20        13.207429
Podium        3.576412
Name: proportion, dtype: float64

In [ ]:
# Identify float64 columns
float_cols = races_data.select_dtypes(include=['float64']).columns

# Convert float64 to int, handling NaN values
for col in float_cols:
    if col in ['points', 'climb_total', 'uci_points', 'length', 'cyclist_age', 'delta']:  # Specify columns that should be int
        races_data[col] = races_data[col].fillna(0).astype(int)  # Replace NaN with 0 before conversion

# Check the new data types
print(races_data.dtypes)

_url                     object
name                     object
points                    int32
uci_points                int32
length                    int32
climb_total             float64
profile                 float64
startlist_quality         int64
average_temperature     float64
date                     object
position                  int64
cyclist_age               int32
is_tarmac                  bool
is_cobbled                 bool
is_gravel                  bool
cyclist_team           category
delta                     int32
cyclist_id               object
position_category        object
dtype: object


In [61]:
# Nationality from object to category
cyclists_data['nationality'] = cyclists_data['nationality'].astype('category')

# Convert the 'cyclist_team' column to category type
races_data['cyclist_team'] = races_data['cyclist_team'].astype('category')
# Convert the 'cyclists_age' column to int32 type
races_data['cyclist_age'] = races_data['cyclist_age'].astype('int32')

# Correcting some type castings that are unnecesary in races dataset
races_data['points'] = races_data['points'].astype(int)
races_data['length'] = races_data['length'].astype(int)
races_data['delta'] = races_data['delta'].astype(int)
races_data['cyclist_age'] = races_data['cyclist_age'].astype(int)
races_data['cyclist_id'] = races_data['cyclist_id'].astype('string')

# Split _url into into ['race_name', 'year', 'stage'] by /
races_data[['race_name', 'year', 'stage']] = races_data['_url'].str.split('/', expand=True)
races_data = races_data.drop(columns=['_url'])
#races_data['stage'] = races_data['stage'].astype('string')
races_data['year'] = races_data['year'].astype(int)

# Now we have 2 columns with race_name
races_data = races_data.drop(columns=['name'])

# Extract month from date and remove date
# Year already from _url and we discard day and hour
races_data['date'] = pd.to_datetime(races_data['date'])
races_data['month'] = races_data['date'].dt.month
races_data = races_data.drop(columns=['date'])

# climb_total and profile are highly correlated, so we combine them
races_data['aug_profile'] = np.where(
    races_data['climb_total'].notna() & races_data['profile'].notna(),
    races_data['climb_total'] / races_data['climb_total'].max() + races_data['profile'],
    np.nan  # Assign NaN if either column is still missing
)
races_data = races_data.drop(columns=['climb_total'])
races_data = races_data.drop(columns=['profile'])

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [ ]:
# Encode categorical values of races df
categorical_cols = ['cyclist_team', 'name', 'stage']
encoder = LabelEncoder()

for col in categorical_cols:
    races_data[col] = encoder.fit_transform(races_data[col])
    
cyclists_data['nationality'] = encoder.fit_transform(cyclists_data['nationality'])

KeyError: 'stage'

In [ ]:
# Always: is_tarmac= true, is_cobbled=False, is_gravel=False
races_data = races_data.drop(columns=['is_tarmac', 'is_cobbled', 'is_gravel'])

## 1.2 Managing missing values

In [ ]:
cyclists_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6134 entries, bruno-surra to ward-vanhoof
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         6134 non-null   object 
 1   birth_year   6121 non-null   float64
 2   weight       3078 non-null   float64
 3   height       3143 non-null   float64
 4   nationality  6133 non-null   object 
dtypes: float64(3), object(2)
memory usage: 287.5+ KB


In [ ]:
# We dont need name with id
cyclists_data = cyclists_data.drop(columns=['name'])

In [ ]:
# Checkimg (and removing if there were any) duplicates in both datasets
duplicated_races = races_data[races_data.duplicated(keep=False)]
print(duplicated_races.shape)

duplicated_cyclists = cyclists_data[cyclists_data.duplicated(keep=False)]
print(duplicated_cyclists.shape)

(0, 14)
(2730, 4)


In [ ]:
print(cyclists_data.shape)
print("Number of null in cyclists dataset")
print("----------------------------------")
print(cyclists_data.isnull().sum(),'\n')

print("Percentage of null in cyclists dataset")
print("----------------------------------")
print(round(cyclists_data.isnull().sum()*100/cyclists_data.shape[0],2).astype(str) + '%')

(6134, 4)
Number of null in cyclists dataset
----------------------------------
birth_year       13
weight         3056
height         2991
nationality       1
dtype: int64 

Percentage of null in cyclists dataset
----------------------------------
birth_year      0.21%
weight         49.82%
height         48.76%
nationality     0.02%
dtype: object


In [ ]:
races_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 589865 entries, tour-de-france/1978/stage-6 to giro-d-italia/2010/stage-1
Data columns (total 14 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   name                 589865 non-null  object 
 1   points               589388 non-null  float64
 2   uci_points           251086 non-null  float64
 3   length               589865 non-null  float64
 4   climb_total          442820 non-null  float64
 5   profile              441671 non-null  float64
 6   startlist_quality    589865 non-null  int64  
 7   average_temperature  29933 non-null   float64
 8   date                 589865 non-null  object 
 9   position             589865 non-null  int64  
 10  cyclist              589865 non-null  object 
 11  cyclist_age          589752 non-null  float64
 12  cyclist_team         430704 non-null  object 
 13  delta                589865 non-null  float64
dtypes: float64(8), int64(2), ob

In [ ]:
print(races_data.shape)
print("Number of null in races dataset")
print("----------------------------------")
print(races_data.isnull().sum(),'\n')

print("Percentage of null in races dataset")
print("----------------------------------")
print(round(races_data.isnull().sum()*100/races_data.shape[0],2).astype(str) + '%')

(589865, 14)
Number of null in races dataset
----------------------------------
name                        0
points                    477
uci_points             338779
length                      0
climb_total            147045
profile                148194
startlist_quality           0
average_temperature    559932
date                        0
position                    0
cyclist                     0
cyclist_age               113
cyclist_team           159161
delta                       0
dtype: int64 

Percentage of null in races dataset
----------------------------------
name                     0.0%
points                  0.08%
uci_points             57.43%
length                   0.0%
climb_total            24.93%
profile                25.12%
startlist_quality        0.0%
average_temperature    94.93%
date                     0.0%
position                 0.0%
cyclist                  0.0%
cyclist_age             0.02%
cyclist_team           26.98%
delta                   

In [ ]:
# It has almost 100% of data loss, so we decided to drop it
races_data = races_data.drop(columns=['average_temperature'])

In [ ]:
# Checking the columns on the cyclists dataset with the most null values
columns_to_plot = ['weight', 'height']

num_columns = len(columns_to_plot)
fig, axes = plt.subplots(num_columns, 1, figsize=(8, num_columns * 4))

for i, column in enumerate(columns_to_plot):
    sns.boxplot(data=cyclists_data, x=column, ax=axes[i])
    axes[i].set_title(f'Boxplot de {column}')
    axes[i].set_xlabel('Valores')

plt.tight_layout() 
plt.show()

In [ ]:
races_data = races_data.rename(columns={"name": "race_name"})
cyclist_data = cyclists_data.rename(columns={"name": "cyclist_name"})

merged_df = races_data.merge(cyclist_data, on="cyclist_id", how="left")

# Mostrar las primeras filas para verificar
print(merged_df.head())